<a href="https://colab.research.google.com/github/JammieBurger/imlo-project-actual/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialising Training and Testing Data.

In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import torch.nn.init as initialize

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
training_data = datasets.Flowers102(
    root="data",
    split='train',
    transform=transform,
    download=True)
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform)

In [10]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size)
test_dataloader = DataLoader(test_data,batch_size)
#for X, y in test_dataloader:
    #print(f"Shape of X [N, C, H, W]: {X.shape}")
    #print(f"Shape of y: {y.shape} {y.dtype}")
    #break
device = ("cuda"if torch.cuda.is_available()
                else "mps"
                if torch.backends.mps.is_available()
                else "cpu")
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.convOne = nn.Conv2d(3,8,(2,2),(1,1),(1,1))
        self.pool = nn.MaxPool2d((2,2),(2,2))
        self.convTwo = nn.Conv2d(8,16,(3,3),(1,1),(1,1))
        self.layerOne = nn.Linear(224*224, 512)
        self.layerTwo = nn.Linear(512,102)
        self.layerThree = nn.Linear(300,200)
        self.layerFour = nn.Linear(200,102)



        self.flatten = nn.Flatten()

        #initialize.xavier_uniform(self.layerOne.weight)
        #initialize.zeros_(self.layerOne.bias)
        #initialize.xavier_uniform(self.layerTwo.weight)
        #initialize.zeros_(self.layerTwo.bias)
        #initialize.xavier_uniform(self.layerThree.weight)
        #initialize.zeros_(self.layerThree.bias)
        #initialize.xavier_uniform(self.layerFour.weight)
        #initialize.zeros_(self.layerFour.bias)


    def forward(self, x):
        x = F.relu(self.convOne(x))
        x = self.pool(x)
        x = F.relu(self.convTwo(x))
        x = self.pool(x)
        x = self.layerOne(x.reshape(x.shape[0],-1))
        #x = self.flatten(x)
        x = self.layerTwo(x)
        #x = self.flatten(x)
        #x = self.layerThree(x)
        #x = self.flatten(x)
        #x = self.layerFour(x)


        return x


Using cpu device


In [11]:
model = NeuralNetwork().to(device)
#print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      pred = model(X)
      loss = loss_fn(pred, y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #optimizer.zero_grad()

      if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Epoch 1
-------------------------------
loss: 4.617416  [   64/ 1020]
Test Error: 
 Accuracy: 4.6%, Avg loss: 4.555111 

Epoch 2
-------------------------------
loss: 5.023941  [   64/ 1020]
Test Error: 
 Accuracy: 7.5%, Avg loss: 4.222493 

Epoch 3
-------------------------------
loss: 4.248496  [   64/ 1020]
Test Error: 
 Accuracy: 9.7%, Avg loss: 4.055971 

Epoch 4
-------------------------------
loss: 3.425990  [   64/ 1020]
Test Error: 
 Accuracy: 12.1%, Avg loss: 3.918230 

Epoch 5
-------------------------------
loss: 2.932298  [   64/ 1020]
Test Error: 
 Accuracy: 14.5%, Avg loss: 3.778499 

Epoch 6
-------------------------------
loss: 2.556329  [   64/ 1020]
Test Error: 
 Accuracy: 16.5%, Avg loss: 3.659967 

Epoch 7
-------------------------------
loss: 2.174614  [   64/ 1020]
Test Error: 
 Accuracy: 18.1%, Avg loss: 3.572287 

Epoch 8
-------------------------------
loss: 1.774557  [   64/ 1020]
Test Error: 
 Accuracy: 18.7%, Avg loss: 3.513673 

Epoch 9
-------------------

KeyboardInterrupt: 